In [5]:
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import dill


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode

API_KEY='Jo8ysFYDZ5-i5jlGul4RCdxXPzLDiTAMAzozMRVTIMLCM7Pf9_2PoGuDxTldodbDTaytnsZwC9cD6UrkYV_puNiS77E3tgTTlS_YoKiI3F8wpnxA6iSkcQO84oi3XHYx' 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
EVENTS_PATH = '/v3/events/'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
#DEFAULT_TERM = 'Reiki Healer'
DEFAULT_LOCATION = 'Manhattan, NY'
SEARCH_LIMIT = 50
OFFSET = 50

url_params = {
    'term': 'restaurant',
    'latitude': '40.810086',
    'longitude' : '-73.921687',
    'radius': '1'
}
print(request(API_HOST, SEARCH_PATH, api_key, url_params=url_params))

NameError: name 'request' is not defined

In [7]:
# -*- coding: utf-8 -*-
"""
Yelp Fusion API code sample.

This program demonstrates the capability of the Yelp Fusion API
by using the Search API to query for businesses by a search term and location,
and the Business API to query additional information about the top result
from the search query.

Please refer to http://www.yelp.com/developers/v3/documentation for the API
documentation.

This program requires the Python requests library, which you can install via:
`pip install -r requirements.txt`.

Sample usage of the program:
`python sample.py --term="bars" --location="San Francisco, CA"`
"""
from __future__ import print_function

import argparse
import json
import pprint
import requests
import sys
import urllib
import dill


# This client code can run on Python 2.x or 3.x.  Your imports can be
# simpler if you only need one of those.
try:
    # For Python 3.0 and later
    from urllib.error import HTTPError
    from urllib.parse import quote
    from urllib.parse import urlencode
except ImportError:
    # Fall back to Python 2's urllib2 and urllib
    from urllib2 import HTTPError
    from urllib import quote
    from urllib import urlencode


# Yelp Fusion no longer uses OAuth as of December 7, 2017.
# You no longer need to provide Client ID to fetch Data
# It now uses private keys to authenticate requests (API Key)
# You can find it on
# https://www.yelp.com/developers/v3/manage_app
API_KEY='Jo8ysFYDZ5-i5jlGul4RCdxXPzLDiTAMAzozMRVTIMLCM7Pf9_2PoGuDxTldodbDTaytnsZwC9cD6UrkYV_puNiS77E3tgTTlS_YoKiI3F8wpnxA6iSkcQO84oi3XHYx' 


# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
EVENTS_PATH = '/v3/events/'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
#DEFAULT_TERM = 'Reiki Healer'
DEFAULT_LOCATION = 'Manhattan, NY'
SEARCH_LIMIT = 50
OFFSET = 50


def request(host, path, api_key, url_params=None):
    """Given your API_KEY, send a GET request to the API.

    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        API_KEY (str): Your API Key.
        url_params (dict): An optional set of query parameters in the request.

    Returns:
        dict: The JSON response from the request.

    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    #print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)
    print(response)

    return response.json()


def search(api_key, term, location, offset):
    """Query the Search API by a search term and location.

    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.

    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'offset': offset,
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit' : SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, api_key, url_params=url_params)

def get_business_reviews(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id + '/reviews'
    print(business_path)
    return request(API_HOST, business_path, api_key, None)


def get_events(api_key, business_id):
    """Query the Business API by a business ID.

    Args:
        business_id (str): The ID of the business to query.

    Returns:
        dict: The JSON response from the request.
    """
    business_path = EVENTS_PATH + business_id

    return request(API_HOST, business_path, api_key, None)

def start(term, locations):
    results = {}
    reviews = {}
    events = {}
    for location in locations:
        try:
            response = search(API_KEY, term, location, 0)
            for business in response['businesses']:
                results[business['id']] = business
                reviews[business['id']] = get_business_reviews(API_KEY, business['id'])
                print(get_business_reviews(API_KEY,business['id']))
                
                events[business['id']] = get_events(API_KEY,business['id'])
                                                     
            print('\nTotal business retrieved: ', len(results))
            print('total in json reply', response['total'])

            cur_offset = OFFSET

            while cur_offset <= (response['total'] + OFFSET):
                response = search(API_KEY, term, location, cur_offset)
                cur_offset += OFFSET
                for business in response['businesses']:
                    results[business['id']] = business
                    reviews[business['id']] = get_business_reviews(API_KEY, business['id'])
                    events[business['id']] = get_events(API_KEY, business['id'])
                    

                print('\nTotal business retrieved: ', len(results))
                print('total in json reply', response['total'])

        except HTTPError as error:
            print('error')
        except:
            print(sys.exc_info())
            
                                                         
    dill.dump(reviews, open('./data/NYC-reviews.dill', 'wb'))    
    dill.dump(events, open('./data/NYC-events.dill', 'wb'))        
    #dill.dump(results, open('./data/NYC-new-restaurant.dill', 'wb'))
                                                         
#start('restaurant', ['10451','10001','11430','11236','10472','11234','11217','10306','11368','10462','11231','11237','10461','11209','10019','11201','11105','10065','11435','10463','10022','10458','10023','10468','10004','11207','10007','10036','11232','11222','11361','11238','11204','11215','11385','11106','11103','10027','10467','11101','10017','11218','11432','10010','10028','10038','11226','11373','10024','11230','11221','10011','10014','10012','11375','10018','10009','11206','10003','11358','11219','10016','11208','10025','10002','10029','10314','11205','10013','11223','11203','11216','11214','11211','10128','10021','11229','11249','11372','10032','11354','11377','10033','11235','11220','11104','11355'])


In [21]:
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
EVENTS_PATH = '/v3/events/'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.


# Defaults for our simple example.
#DEFAULT_TERM = 'Reiki Healer'
DEFAULT_LOCATION = 'Manhattan, NY'
SEARCH_LIMIT = 50
OFFSET = 50
api_key='Jo8ysFYDZ5-i5jlGul4RCdxXPzLDiTAMAzozMRVTIMLCM7Pf9_2PoGuDxTldodbDTaytnsZwC9cD6UrkYV_puNiS77E3tgTTlS_YoKiI3F8wpnxA6iSkcQO84oi3XHYx' 


url_params = {
    'term': 'restaurant',
    'latitude': '40.807370',
    'longitude': '-73.927520',
    'radius': '1700',
    'limit' : 50,
    'offset' : 0
}
print(request(API_HOST, SEARCH_PATH, api_key, url_params=url_params))

<Response [200]>
{'businesses': [{'id': 'g_Zz87qghqq9bKHiKyws0w', 'alias': 'beatstro-bronx', 'name': 'Beatstro', 'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/1aDAmkaztpdSyeyrIH3ePQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/beatstro-bronx?adjust_creative=hyX7aOMDHtCU0xWoARemvw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hyX7aOMDHtCU0xWoARemvw', 'review_count': 158, 'categories': [{'alias': 'tradamerican', 'title': 'American (Traditional)'}, {'alias': 'speakeasies', 'title': 'Speakeasies'}, {'alias': 'latin', 'title': 'Latin American'}], 'rating': 4.0, 'coordinates': {'latitude': 40.80737, 'longitude': -73.92752}, 'transactions': ['delivery', 'pickup', 'restaurant_reservation'], 'price': '$$', 'location': {'address1': '135 Alexander Ave', 'address2': None, 'address3': None, 'city': 'Bronx', 'zip_code': '10454', 'country': 'US', 'state': 'NY', 'display_address': ['135 Alexander Ave', 'Bronx, NY 10454']}, 'phone': '+17184899397', 'displ

In [3]:
from googleapiclient.discovery import build

my_api_key = ''

def google_search(search_term, api_key, cse_id, **kwargs):
    service = build("customsearch", "v1", developerKey=api_key)
    res = service.cse().list(q=search_term, cx=cse_id, **kwargs).execute()
    return res['items']

results = google_search(
    'restaurant in Boston, MA', my_api_key, my_cse_id, num=10)
for result in results:
    print(result)
    

ModuleNotFoundError: No module named 'googleapiclient'

In [4]:
rest=dill.load(open('./data/restaurant-NYC.dill', 'rb'))
small={}
for item in rest:
    if len(small) <=10:
        small[item] = rest[item]
    
dill.dump(small, open('./data/restaurant-NYC-smaller.dill', 'wb'))


In [5]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
        return dict(items)
        
    return [flatten(dict(attr[0]), parent_key='', sep='_') for attr in X]

In [4]:
import dill
import numpy as np
import operator

rest=dill.load(open('./data/restaurant-NYC.dill', 'rb'))
counts={}
for restrau in rest:
    if rest[restrau]['name'] in counts:
        counts[rest[restrau]['name']] += 1
    else:
        counts[rest[restrau]['name']] = 1
    if rest[restrau]['alias'] in counts:
        counts[rest[restrau]['alias']] += 1
    else:
        counts[rest[restrau]['alias']] = 1

        
print(sorted(counts.items(), key=operator.itemgetter(1)))
dill.dump(counts, open('./data/restaurant-is_closed.dill', 'wb'))
        


[('Belle Harlem', 1), ('belle-harlem-new-york', 1), ('Renaissance Harlem', 1), ('renaissance-harlem-new-york', 1), ('Beatstro', 1), ('beatstro-bronx', 1), ('Gun Hill Tavern', 1), ('gun-hill-tavern-new-york', 1), ('Nobodys Pizza', 1), ('nobodys-pizza-bronx', 1), ('Porto Salvo', 1), ('porto-salvo-bronx', 1), ("Charlie's Bar & Kitchen", 1), ('charlies-bar-and-kitchen-bronx', 1), ('Red Rooster Harlem', 1), ('red-rooster-harlem-new-york-3', 1), ('Xochimilco Family Restaurant', 1), ('xochimilco-family-restaurant-bronx', 1), ('Mottley Kitchen', 1), ('mottley-kitchen-south-bronx', 1), ('Milk Burger', 1), ('milk-burger-bronx', 1), ('Babbalucci', 1), ('babbalucci-new-york', 1), ('Barepas', 1), ('barepas-new-york', 1), ('Los Nisperos', 1), ('los-nisperos-new-york', 1), ('Landin Mac & Cheese', 1), ('landin-mac-and-cheese-bronx', 1), ("Sisters' Caribbean Cuisine", 1), ('sisters-caribbean-cuisine-new-york', 1), ('The Row Harlem', 1), ('the-row-harlem-new-york', 1), ('Benyam Cuisine', 1), ('benyam-cu

In [ ]:
import dill
import numpy as np

rest=dill.load(open('./data/restaurant-NYC.dill', 'rb'))
restaurant_counts=dill.load(open('./data/restaurant-is_closed.dill', 'rb'))

X_train = []
Y_train = []
features = ['review_count', 'price', 'transactions', 'is_closed', 'is_chain']
categories = {}
count_price = 0
count_no_price = 0
count_attr = 0
count_no_attr = 0
for restrau in rest:    
    cats = np.array([])
    for eachCat in rest[restrau]['categories']:
        if eachCat['alias'] not in categories:
            categories[eachCat['alias']] = len(categories) 
    '''
    if 'attributes' in rest[restrau]:
        print(flatten(rest[restrau]['attributes']))
        count_attr+=1
    else:
        count_no_attr+=1
    '''
    
print(features)
print(categories)
count0 = 0
count1=0
count=0
for restrau in rest:
    rest_name = rest[restrau]['name']
    rest_alias = rest[restrau]['alias']
    oneRest = []
    oneRest.append(rest[restrau]['review_count'])
    if 'price' in rest[restrau]:
        oneRest.append(len(rest[restrau]['price']))
    else:
        continue
    oneRest.append(len(list(rest[restrau]['transactions'])))
    

    if 'is_closed' in rest[restrau]:
        if rest[restrau]['is_closed'] is True:
            oneRest.append(1)
            count1+=1
        else:
            print(rest[restrau])
            oneRest.append(0)
            count0+=1
    else:
        count+=1
        continue
    
    if (rest_name in restaurant_counts and restaurant_counts[rest_name] > 0) or (rest_alias in restaurant_counts and restaurant_counts[rest_alias] > 0):
        oneRest.append(1)
    else:
        oneRest.append(0)
    
    cats = np.zeros(len(categories))
    for eachCat in rest[restrau]['categories']:
        cats[int(categories[eachCat['alias']])] = 1
    oneRest.extend(cats)
    #print(oneRest)
    #reviews[rest[restrau]]
    Y_train.append(rest[restrau]['rating'])
    #print(oneRest)
    X_train.append(oneRest)

print("count0" + str(count0))
print("count1" +str(count1))
print("count" + str(count))


dill.dump(X_train, open('./data/X_train4.dill', 'wb'))
dill.dump(Y_train, open('./data/Y_train4.dill', 'wb'))

        

    
    


['review_count', 'price', 'transactions', 'is_closed', 'is_chain']
{'newamerican': 0, 'southern': 1, 'comfortfood': 2, 'tradamerican': 3, 'speakeasies': 4, 'latin': 5, 'bars': 6, 'italian': 7, 'pizza': 8, 'chicken_wings': 9, 'wine_bars': 10, 'cocktailbars': 11, 'mexican': 12, 'cafes': 13, 'coffee': 14, 'catering': 15, 'burgers': 16, 'salad': 17, 'venezuelan': 18, 'tapasmallplates': 19, 'peruvian': 20, 'caribbean': 21, 'soulfood': 22, 'lounges': 23, 'breakfast_brunch': 24, 'desserts': 25, 'ethiopian': 26, 'spanish': 27, 'puertorican': 28, 'seafood': 29, 'vegetarian': 30, 'beerbar': 31, 'sportsbars': 32, 'tapas': 33, 'french': 34, 'senegalese': 35, 'juicebars': 36, 'indpak': 37, 'soup': 38, 'dominican': 39, 'steak': 40, 'sushi': 41, 'asianfusion': 42, 'thai': 43, 'chinese': 44, 'japanese': 45, 'foodstands': 46, 'buffets': 47, 'mediterranean': 48, 'korean': 49, 'ramen': 50, 'sandwiches': 51, 'african': 52, 'waffles': 53, 'noodles': 54, 'fishnchips': 55, 'honduran': 56, 'salvadoran': 57, '

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



{'id': 'wwiqriIMSiMlyod1-kdDdg', 'alias': 'central-perk-deli-elizabeth', 'name': 'Central Perk Deli', 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/-Kp9jXh5LXEhoRgkaoqWFQ/o.jpg', 'is_closed': False, 'url': 'https://www.yelp.com/biz/central-perk-deli-elizabeth?adjust_creative=hyX7aOMDHtCU0xWoARemvw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=hyX7aOMDHtCU0xWoARemvw', 'review_count': 9, 'categories': [{'alias': 'delis', 'title': 'Delis'}], 'rating': 5.0, 'coordinates': {'latitude': 40.6623860538933, 'longitude': -74.2369668136918}, 'transactions': [], 'price': '$', 'location': {'address1': '804 Pennington St', 'address2': '', 'address3': '', 'city': 'Elizabeth', 'zip_code': '07202', 'country': 'US', 'state': 'NJ', 'display_address': ['804 Pennington St', 'Elizabeth, NJ 07202']}, 'phone': '+19083554055', 'display_phone': '(908) 355-4055', 'distance': 9666.350471544687}
{'id': 'Q40CIMN_KtlZU9vViRITyA', 'alias': 'garden-restaurant-union', 'name': 'Garden Res

In [ ]:
X_training = dill.load(open('./data/X_train2.dill', 'rb'))
Y_training = dill.load(open('./data/Y_train2.dill', 'rb'))


for item in X_training:
    

## 

In [7]:
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction import DictVectorizer
from sklearn import base

class MyDualEstimator(base.BaseEstimator, base.TransformerMixin):
   def __init__(self, linear_model, non_linear_model):
       self.model_1 = linear_model
       self.model_2 = non_linear_model
   
   def fit(self, X, y):
       # model_1.fit(X,y)
       self.model_1.fit(X,y)
       
       # residual = model_1.predict - y
       residual = y - self.model_1.predict(X)
       
       # model_2.fit(X, residual)
       self.model_2.fit(X, residual)
       return self        
   
   def predict(self, X):
       return self.model_1.predict(X) + self.model_2.predict(X)

DualEstimator = MyDualEstimator(Ridge(alpha=10), RandomForestRegressor(n_estimators=100))

attribute_est = Pipeline([      # DictVectorizer
                        ('Estimator', DualEstimator)     # Estimator: Ridge + RandomForest
                       ])
from sklearn.utils import shuffle

data, star_ratings = shuffle(X_train, Y_train)
attribute_est.fit(X_train[0:15000], Y_train[0:15000])
results = attribute_est.predict(X_train[15000:])




In [8]:
print(results)

[3.88286344 3.99006173 3.91819678 ... 3.09097206 2.72671587 4.05098462]


In [9]:
#print(Y_train[:20000])
index_results= 0 
print(len(Y_train))
for index in range(15000, len(Y_train)):
    print((Y_train[index] - results[index_results]) * 100.0 / Y_train[index])
    index+=1
    index_results+=1
    

17537
2.928413962074472
0.24845666791495358
2.045080483435946
8.885174222543617
-16.2004568848805
-8.16191715224143
6.440932630020613
-12.490074836793383
-1.6274796750275276
-0.07752207156523916
-8.33529998108679
-6.418059893925194
22.893605175740905
-15.036201061294971
-18.84605441615445
14.258112486481217
5.747038850007158
-3.537821239605408
-0.9151097202328762
-16.795197922952543
13.330934011163352
10.901325771105379
8.968782995114053
3.5018362941822874
0.1780513559753505
2.5760596433515137
-5.110883107678723
-35.98098659575264
20.781386756617298
2.8985916432470726
-11.977037903498326
-23.529210497731658
-9.779458646167123
5.987344828979576
5.2025912697233645
13.058740006396475
24.046818419495096
7.560953416631899
-4.797954834324335
-10.31811011040844
13.279040656463204
-14.274999890300423
-11.275342462658307
15.013533549027757
-18.603229492408754
1.5054108358656393
-6.6105426407044705
-11.463734783141012
-5.868826854598774
7.434801545968128
-19.391825470072405
6.806979523559775
38.

10.771130592485534
-1.5094949307956407
-4.464502181947237
-49.80538958547374
-20.065408722451973
-0.15757848401853902
31.00081908831909
11.045426594794046
-19.4310690147962
16.549707781100732
-23.34288073284362
-3.513786233751683
9.108065150251221
-45.561913868038616
8.42151916054652
-14.513174553318262
3.0537021335547188
-2.332983540236671
7.966955952707356
1.6731640690292362
4.4147933817629825
5.465699361869004
19.651325898300012
19.844721132882803
-26.22818817125072
2.6211638562537942
-93.62179929280634
4.96123636831366
-34.473381246843864
2.1307645290645865
5.051176085858773
-113.56586834797179
-50.30208117810702
-13.805168102255168
10.96500957112223
-5.966420619186012
5.134282268578383
37.48007103892849
21.9497000686064
27.870139582639577
3.35554478969462
3.239587727145426
12.577430555555559
-27.009935281237066
38.8756913416854
33.80429154179154
14.8965561282545
-11.604416446931065
-10.324922793298352
0.0732711619549496
-10.717796780495966
40.424214429061855
-12.47156698739039
1.1